In [1]:
import pandas as pd
import numpy as np
import re
from datetime import datetime

#Se lee la base de datos
dbase=pd.read_csv("tablero_solicitudes_resumen_30-mar-2022.csv")
dbase.head()


,Instancia,Matrícula,Nombre,Ap Paterno,Ap Materno,Campus,Programa,Promedio,Núm. MatAprobadas,Núm. MatTotales,...,Periodo Solicitud,OportunidadesSeleccionadas,OportunidadesAprobadas,Estatus,OportunidadAsignada,Nombre Oportunidad Asignada,CampusAdministrador,PeriodoAcadémico,Actividad Actual,Fecha
0,8453,A01139867,Andrés Gerardo,Garza,Canales,Monterrey,INT11,90.491,57.0,57.0,...,Enero - Mayo 2015,1 - EUA-5INT-020V,EUA-5INT-020V,Asignada,EUA-5INT-020V,California State University - Intercambio Inte...,Querétaro,Enero - Mayo 2015,Resultado de Asignación,20/AGO/2014 02:22PM
1,8289,A01018302,Pablo,Zebadúa,Vega,Santa Fe,ITC11,85.684,0.0,57.0,...,Enero - Mayo 2015,"1 - SIN-5INT-003,<br \>3 - CHN-5INT-003,<br \>...","SIN-5INT-003,<br \> JAP-5INT-002A,<br \> CHN-5...",Asignada,JAP-5INT-002A,Hokkaido University - Intercambio Internaciona...,Monterrey,Enero - Mayo 2015,Resultado de Asignación,29/AGO/2014 04:34PM
2,8510,A01205500,Laura,Treviño,Hernández,Querétaro,ISC11,89.810,0.0,57.0,...,Enero - Mayo 2015,"1 - CZE-5INT-001,<br \>2 - FIN-5INT-004A,<br \...","CZE-5INT-001,<br \> FIN-5INT-004A,<br \> FIN-5...",Asignada,CZE-5INT-001,Czech Technical University - Intercambio Inte...,Monterrey,Enero - Mayo 2015,Resultado de Asignación,29/AGO/2014 04:35PM
3,8007,A01139036,Arturo Mauricio,Lara,Martínez,Monterrey,CPF11,85.465,0.0,56.0,...,Enero - Mayo 2015,"1 - ALE-5INT-003,<br \>5 - ALE-5INT-034A,<br \...","ALE-5INT-003,<br \> HOL-5INT-018,<br \> FRA-5I...",Asignada,HOL-5INT-018,Universiteit Maastricht - Intercambio Internac...,Guadalajara,Enero - Mayo 2015,Resultado de Asignación,29/AGO/2014 04:43PM
4,14315,A00757284,Ana Gabriela,Amaro,Rodríguez,Chihuahua,LMC11,84.523,0.0,59.0,...,Enero - Mayo 2015,"1 - ESP-5INT-057B,<br \>3 - ESP-5INT-059,<br \...","ESP-5INT-057B,<br \> ESP-5INT-062A,<br \> ESP-...",Asignada,ARG-5INT-015,Universidad de Ciencias Empresariales y Social...,Ciudad de México,Enero - Mayo 2015,Resultado de Asignación,29/AGO/2014 04:43PM


Limpieza de datos

In [2]:
#Se eliminan las columnas que no se van a utilizar
dbase=dbase.drop(['Nombre','Ap Paterno','Ap Materno','Núm. MatAprobadas','Núm. MatTotales','Periodo Solicitud','OportunidadesAprobadas','Actividad Actual','CampusAdministrador'], axis=1)

#Se eliminan filas con campos vacíos 
#Solo se deben conservar solicitudes aceptadas así que es importante eliminar nulos en OportunidadAsignada, pues las oportunidades rechazadas están vacías en este campo.
dbase=dbase.dropna(subset=['Matrícula', 'Campus','Programa','Promedio','PuntajeAsignación','Nivel','OportunidadesSeleccionadas','Estatus','OportunidadAsignada']).reset_index(drop=True)

dbase

,Instancia,Matrícula,Campus,Programa,Promedio,PuntajeAsignación,Nivel,OportunidadesSeleccionadas,Estatus,OportunidadAsignada,Nombre Oportunidad Asignada,PeriodoAcadémico,Fecha
0,8453,A01139867,Monterrey,INT11,90.491,83.453,Profesional,1 - EUA-5INT-020V,Asignada,EUA-5INT-020V,California State University - Intercambio Inte...,Enero - Mayo 2015,20/AGO/2014 02:22PM
1,8289,A01018302,Santa Fe,ITC11,85.684,77.555,Profesional,"1 - SIN-5INT-003,<br \>3 - CHN-5INT-003,<br \>...",Asignada,JAP-5INT-002A,Hokkaido University - Intercambio Internaciona...,Enero - Mayo 2015,29/AGO/2014 04:34PM
2,8510,A01205500,Querétaro,ISC11,89.810,85.757,Profesional,"1 - CZE-5INT-001,<br \>2 - FIN-5INT-004A,<br \...",Asignada,CZE-5INT-001,Czech Technical University - Intercambio Inte...,Enero - Mayo 2015,29/AGO/2014 04:35PM
3,8007,A01139036,Monterrey,CPF11,85.465,81.528,Profesional,"1 - ALE-5INT-003,<br \>5 - ALE-5INT-034A,<br \...",Asignada,HOL-5INT-018,Universiteit Maastricht - Intercambio Internac...,Enero - Mayo 2015,29/AGO/2014 04:43PM
4,14315,A00757284,Chihuahua,LMC11,84.523,78.921,Profesional,"1 - ESP-5INT-057B,<br \>3 - ESP-5INT-059,<br \...",Asignada,ARG-5INT-015,Universidad de Ciencias Empresariales y Social...,Enero - Mayo 2015,29/AGO/2014 04:43PM
...,...,...,...,...,...,...,...,...,...,...,...,...,...
48997,589572,A01365530,Santa Fe,LEM11,92.326,89.780,Profesional,1 - ITA-V5SA-020M,Pre-Asignada - Pendiente pago,ITA-V5SA-020M,Università Cattolica del Sacro Cuore - Study A...,Verano 2022 Profesional,30/Mar/2022 5:01 PM
48998,577669,A01561940,Chihuahua,IMA11,92.024,86.391,Profesional,1 - NOR-5INT-001,Admitido,NOR-5INT-001,BI Norwegian Business School - Intercambio Int...,Agosto - Diciembre 2022,30/Mar/2022 5:10 PM
48999,579143,A01283470,Monterrey,LDI17,87.648,82.700,Profesional,1 - ITA-5CE-016E,Asignado,ITA-5CE-016E,Scuola Lorenzo de' Medici - Certificado de Esp...,Agosto - Diciembre 2022,30/Mar/2022 5:19 PM
49000,589315,A01154926,Monterrey,IFI11,86.714,85.374,Profesional,1 - FRA-V5SA-066C,Asignado,FRA-V5SA-066C,Excelia Group - La Rochelle - Study Abroad Tra...,Verano 2022 Profesional,30/Mar/2022 5:25 PM


In [3]:
#Se analiza si hay alumnos con registros repetidos
print(len(dbase['Instancia'].unique())) #Las instancias no se repiten
print(len(dbase['Matrícula'].unique())) #Las matriculas se repiten, pues no es la misma cantidad que las instancias

#Por lo tanto, existen registros repetidos

49002
37551


In [4]:
#No queremos estudiantes repetidos, si ya se fueron más de una vez solo se debe conservar el ultimo registro
#Se borran las celdas duplicadas que tengan misma matrícula
#Las celdas ya vienen ordenadas por fecha(comenzando del más antiguo), con keep=last solo se conserva el duplicado más reciente
dbase=dbase.drop_duplicates(subset=['Matrícula'], keep='last').reset_index(drop=True)
dbase

,Instancia,Matrícula,Campus,Programa,Promedio,PuntajeAsignación,Nivel,OportunidadesSeleccionadas,Estatus,OportunidadAsignada,Nombre Oportunidad Asignada,PeriodoAcadémico,Fecha
0,8453,A01139867,Monterrey,INT11,90.491,83.453,Profesional,1 - EUA-5INT-020V,Asignada,EUA-5INT-020V,California State University - Intercambio Inte...,Enero - Mayo 2015,20/AGO/2014 02:22PM
1,8289,A01018302,Santa Fe,ITC11,85.684,77.555,Profesional,"1 - SIN-5INT-003,<br \>3 - CHN-5INT-003,<br \>...",Asignada,JAP-5INT-002A,Hokkaido University - Intercambio Internaciona...,Enero - Mayo 2015,29/AGO/2014 04:34PM
2,8510,A01205500,Querétaro,ISC11,89.810,85.757,Profesional,"1 - CZE-5INT-001,<br \>2 - FIN-5INT-004A,<br \...",Asignada,CZE-5INT-001,Czech Technical University - Intercambio Inte...,Enero - Mayo 2015,29/AGO/2014 04:35PM
3,8007,A01139036,Monterrey,CPF11,85.465,81.528,Profesional,"1 - ALE-5INT-003,<br \>5 - ALE-5INT-034A,<br \...",Asignada,HOL-5INT-018,Universiteit Maastricht - Intercambio Internac...,Enero - Mayo 2015,29/AGO/2014 04:43PM
4,14315,A00757284,Chihuahua,LMC11,84.523,78.921,Profesional,"1 - ESP-5INT-057B,<br \>3 - ESP-5INT-059,<br \...",Asignada,ARG-5INT-015,Universidad de Ciencias Empresariales y Social...,Enero - Mayo 2015,29/AGO/2014 04:43PM
...,...,...,...,...,...,...,...,...,...,...,...,...,...
37546,589572,A01365530,Santa Fe,LEM11,92.326,89.780,Profesional,1 - ITA-V5SA-020M,Pre-Asignada - Pendiente pago,ITA-V5SA-020M,Università Cattolica del Sacro Cuore - Study A...,Verano 2022 Profesional,30/Mar/2022 5:01 PM
37547,577669,A01561940,Chihuahua,IMA11,92.024,86.391,Profesional,1 - NOR-5INT-001,Admitido,NOR-5INT-001,BI Norwegian Business School - Intercambio Int...,Agosto - Diciembre 2022,30/Mar/2022 5:10 PM
37548,579143,A01283470,Monterrey,LDI17,87.648,82.700,Profesional,1 - ITA-5CE-016E,Asignado,ITA-5CE-016E,Scuola Lorenzo de' Medici - Certificado de Esp...,Agosto - Diciembre 2022,30/Mar/2022 5:19 PM
37549,589315,A01154926,Monterrey,IFI11,86.714,85.374,Profesional,1 - FRA-V5SA-066C,Asignado,FRA-V5SA-066C,Excelia Group - La Rochelle - Study Abroad Tra...,Verano 2022 Profesional,30/Mar/2022 5:25 PM


In [5]:
#Se quiere conservar registros asignados, se analizan las distintas variantes
freq = dbase['Estatus'].value_counts() 
print(freq) 

Asignada                                          5132
Asignada - Documentos aprobados                   4736
Admitido                                          3952
Asignada - Materias aprobadas                     2940
Admitido - Documentos aprobados                   2863
Asignada - Pendiente registrar materias           2534
Admitido - Documentos de Salida                   2086
En extranjero - Materias aprobadas                1915
En extranjero - Pendiente registrar materias      1857
Asignada - Documentos enviados a aprobación       1502
Asignado                                          1446
Asignada - Pendiente enviar documentos             867
Admitido - Documentos enviados a aprobación        826
Asignada - Materias enviadas a aprobación          639
En extranjero - Regreso enviado                    630
En extranjero - Materias en aprobación             612
Pre-Asignada - Pendiente pago                      398
Asignada - Documentos rechazados                   397
Asignada -

In [6]:
#Se filtra la columna Estatus y se crea una nueva, solo se llenan las filas que contengan Admitido,Asignada y Asignada, el resto quedan vacíos
dbase['Estatusfiltrado']=[" ".join(re.findall('^Admitido|^Asignad[ao]',s)) for s in dbase['Estatus']]

#Se eliminan los registros que hayan quedado vacíos de la nueva columna
dbase=dbase.loc[dbase['Estatusfiltrado'] != ''].reset_index(drop=True)
dbase

,Instancia,Matrícula,Campus,Programa,Promedio,PuntajeAsignación,Nivel,OportunidadesSeleccionadas,Estatus,OportunidadAsignada,Nombre Oportunidad Asignada,PeriodoAcadémico,Fecha,Estatusfiltrado
0,8453,A01139867,Monterrey,INT11,90.491,83.453,Profesional,1 - EUA-5INT-020V,Asignada,EUA-5INT-020V,California State University - Intercambio Inte...,Enero - Mayo 2015,20/AGO/2014 02:22PM,Asignada
1,8289,A01018302,Santa Fe,ITC11,85.684,77.555,Profesional,"1 - SIN-5INT-003,<br \>3 - CHN-5INT-003,<br \>...",Asignada,JAP-5INT-002A,Hokkaido University - Intercambio Internaciona...,Enero - Mayo 2015,29/AGO/2014 04:34PM,Asignada
2,8510,A01205500,Querétaro,ISC11,89.810,85.757,Profesional,"1 - CZE-5INT-001,<br \>2 - FIN-5INT-004A,<br \...",Asignada,CZE-5INT-001,Czech Technical University - Intercambio Inte...,Enero - Mayo 2015,29/AGO/2014 04:35PM,Asignada
3,8007,A01139036,Monterrey,CPF11,85.465,81.528,Profesional,"1 - ALE-5INT-003,<br \>5 - ALE-5INT-034A,<br \...",Asignada,HOL-5INT-018,Universiteit Maastricht - Intercambio Internac...,Enero - Mayo 2015,29/AGO/2014 04:43PM,Asignada
4,14315,A00757284,Chihuahua,LMC11,84.523,78.921,Profesional,"1 - ESP-5INT-057B,<br \>3 - ESP-5INT-059,<br \...",Asignada,ARG-5INT-015,Universidad de Ciencias Empresariales y Social...,Enero - Mayo 2015,29/AGO/2014 04:43PM,Asignada
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31364,588529,A01422658,Cuernavaca,LAE16,84.880,81.964,Profesional,"1 - ITA-V5SA-020M,<br \>2 - ITA-V5SA-020E,<br ...",Asignado,ITA-V5SA-020M,Università Cattolica del Sacro Cuore - Study A...,Verano 2022 Profesional,30/Mar/2022 4:33 PM,Asignado
31365,549639,A01561726,Chihuahua,ARQ11,84.511,109.763,Profesional,"1 - ESP-5PP-001B,<br \>2 - ESP-5INT-064A,<br \...",Admitido,ESP-5PP-001B,The Intern Group-Madrid - Prácticas Profesiona...,Agosto - Diciembre 2022,30/Mar/2022 4:34 PM,Admitido
31366,577669,A01561940,Chihuahua,IMA11,92.024,86.391,Profesional,1 - NOR-5INT-001,Admitido,NOR-5INT-001,BI Norwegian Business School - Intercambio Int...,Agosto - Diciembre 2022,30/Mar/2022 5:10 PM,Admitido
31367,579143,A01283470,Monterrey,LDI17,87.648,82.700,Profesional,1 - ITA-5CE-016E,Asignado,ITA-5CE-016E,Scuola Lorenzo de' Medici - Certificado de Esp...,Agosto - Diciembre 2022,30/Mar/2022 5:19 PM,Asignado


In [7]:
#Se comprueba que el filtro funciona
freq = dbase['Estatus'].value_counts() 
print(freq) 

Asignada                                       5132
Asignada - Documentos aprobados                4736
Admitido                                       3952
Asignada - Materias aprobadas                  2940
Admitido - Documentos aprobados                2863
Asignada - Pendiente registrar materias        2534
Admitido - Documentos de Salida                2086
Asignada - Documentos enviados a aprobación    1502
Asignado                                       1446
Asignada - Pendiente enviar documentos          867
Admitido - Documentos enviados a aprobación     826
Asignada - Materias enviadas a aprobación       639
Asignada - Documentos rechazados                397
Asignada - Materias en aprobación               382
Asignada - Documentos enviados                  330
Asignada - Documentos en aprobación             286
Admitido - Documentos rechazados                124
Asignada - Materias rechazadas                  104
Admitido - Documentos en aprobación             100
Asignada - M

In [8]:
#Dato de prueba para filtrado
prueba=dbase.iloc[14,7]
prueba

'2 - ALE-5INT-013,<br \\>3 - ALE-5INT-105,<br \\>4 - AUS-5INT-024,<br \\>6 - ESP-5INT-062A,<br \\>1 - ALE-5INT-138,<br \\>7 - ESP-5INT-100E,<br \\>5 - AUS-5INT-049B'

In [9]:
#Se prueba el filtrado del dato para solo tomar la primera opción
l1=re.findall('1 - (\w{,3}[\s-]\w{,4}[\s-]\w{,4})',prueba)
l1

['ALE-5INT-138']

In [10]:
#Se aplica el filtrado a toda la columna, solo se obtiene la primera opción que selecciono la persona
dbase['Primeraopcion']=[" ".join(re.findall('1 - (\w{,3}[\s-]\w{,4}[\s-]\w{,4})',s)) for s in dbase['OportunidadesSeleccionadas']]
dbase=dbase.loc[dbase['Primeraopcion'] != ''].reset_index(drop=True)


In [11]:
#Se obtiene el numero de opción asignada al alumno
dbase['OportunidadAsignada'] = dbase['OportunidadAsignada'].astype(str)
dbase['OpcionAsignada']=[" ".join(re.findall(rf"([0-9]+) - {b}",a)) for a, b in zip(dbase['OportunidadesSeleccionadas'], dbase['OportunidadAsignada']) ]
dbase


,Instancia,Matrícula,Campus,Programa,Promedio,PuntajeAsignación,Nivel,OportunidadesSeleccionadas,Estatus,OportunidadAsignada,Nombre Oportunidad Asignada,PeriodoAcadémico,Fecha,Estatusfiltrado,Primeraopcion,OpcionAsignada
0,8453,A01139867,Monterrey,INT11,90.491,83.453,Profesional,1 - EUA-5INT-020V,Asignada,EUA-5INT-020V,California State University - Intercambio Inte...,Enero - Mayo 2015,20/AGO/2014 02:22PM,Asignada,EUA-5INT-020V,1
1,8289,A01018302,Santa Fe,ITC11,85.684,77.555,Profesional,"1 - SIN-5INT-003,<br \>3 - CHN-5INT-003,<br \>...",Asignada,JAP-5INT-002A,Hokkaido University - Intercambio Internaciona...,Enero - Mayo 2015,29/AGO/2014 04:34PM,Asignada,SIN-5INT-003,2
2,8510,A01205500,Querétaro,ISC11,89.810,85.757,Profesional,"1 - CZE-5INT-001,<br \>2 - FIN-5INT-004A,<br \...",Asignada,CZE-5INT-001,Czech Technical University - Intercambio Inte...,Enero - Mayo 2015,29/AGO/2014 04:35PM,Asignada,CZE-5INT-001,1
3,8007,A01139036,Monterrey,CPF11,85.465,81.528,Profesional,"1 - ALE-5INT-003,<br \>5 - ALE-5INT-034A,<br \...",Asignada,HOL-5INT-018,Universiteit Maastricht - Intercambio Internac...,Enero - Mayo 2015,29/AGO/2014 04:43PM,Asignada,ALE-5INT-003,2
4,14315,A00757284,Chihuahua,LMC11,84.523,78.921,Profesional,"1 - ESP-5INT-057B,<br \>3 - ESP-5INT-059,<br \...",Asignada,ARG-5INT-015,Universidad de Ciencias Empresariales y Social...,Enero - Mayo 2015,29/AGO/2014 04:43PM,Asignada,ESP-5INT-057B,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30591,588529,A01422658,Cuernavaca,LAE16,84.880,81.964,Profesional,"1 - ITA-V5SA-020M,<br \>2 - ITA-V5SA-020E,<br ...",Asignado,ITA-V5SA-020M,Università Cattolica del Sacro Cuore - Study A...,Verano 2022 Profesional,30/Mar/2022 4:33 PM,Asignado,ITA-V5SA-020M,1
30592,549639,A01561726,Chihuahua,ARQ11,84.511,109.763,Profesional,"1 - ESP-5PP-001B,<br \>2 - ESP-5INT-064A,<br \...",Admitido,ESP-5PP-001B,The Intern Group-Madrid - Prácticas Profesiona...,Agosto - Diciembre 2022,30/Mar/2022 4:34 PM,Admitido,ESP-5PP-001B,1
30593,577669,A01561940,Chihuahua,IMA11,92.024,86.391,Profesional,1 - NOR-5INT-001,Admitido,NOR-5INT-001,BI Norwegian Business School - Intercambio Int...,Agosto - Diciembre 2022,30/Mar/2022 5:10 PM,Admitido,NOR-5INT-001,1
30594,579143,A01283470,Monterrey,LDI17,87.648,82.700,Profesional,1 - ITA-5CE-016E,Asignado,ITA-5CE-016E,Scuola Lorenzo de' Medici - Certificado de Esp...,Agosto - Diciembre 2022,30/Mar/2022 5:19 PM,Asignado,ITA-5CE-016E,1


In [12]:
#Se observa que la nueva columna tiene datos atípicos
freq = dbase['OpcionAsignada'].value_counts() 
print(freq) 

1        27064
2         1423
3          726
           445
4          440
5          234
6          156
7           84
1 2         13
3 4          2
1 4          2
1 3 5        1
3 6          1
1 3          1
2 4          1
2 3          1
3 5          1
2 1          1
Name: OpcionAsignada, dtype: int64


In [13]:
#Se eliminan datos atípicos y de los registros que cuentan con 2 numeros solo se toma el primero
dbase=dbase.drop(dbase[dbase['OpcionAsignada']==''].index).reset_index(drop=True)
dbase['OpcionAsignada']=[(a[0]) for a in dbase['OpcionAsignada']]
dbase['OpcionAsignada'] = pd.to_numeric(dbase['OpcionAsignada'])
dbase

,Instancia,Matrícula,Campus,Programa,Promedio,PuntajeAsignación,Nivel,OportunidadesSeleccionadas,Estatus,OportunidadAsignada,Nombre Oportunidad Asignada,PeriodoAcadémico,Fecha,Estatusfiltrado,Primeraopcion,OpcionAsignada
0,8453,A01139867,Monterrey,INT11,90.491,83.453,Profesional,1 - EUA-5INT-020V,Asignada,EUA-5INT-020V,California State University - Intercambio Inte...,Enero - Mayo 2015,20/AGO/2014 02:22PM,Asignada,EUA-5INT-020V,1
1,8289,A01018302,Santa Fe,ITC11,85.684,77.555,Profesional,"1 - SIN-5INT-003,<br \>3 - CHN-5INT-003,<br \>...",Asignada,JAP-5INT-002A,Hokkaido University - Intercambio Internaciona...,Enero - Mayo 2015,29/AGO/2014 04:34PM,Asignada,SIN-5INT-003,2
2,8510,A01205500,Querétaro,ISC11,89.810,85.757,Profesional,"1 - CZE-5INT-001,<br \>2 - FIN-5INT-004A,<br \...",Asignada,CZE-5INT-001,Czech Technical University - Intercambio Inte...,Enero - Mayo 2015,29/AGO/2014 04:35PM,Asignada,CZE-5INT-001,1
3,8007,A01139036,Monterrey,CPF11,85.465,81.528,Profesional,"1 - ALE-5INT-003,<br \>5 - ALE-5INT-034A,<br \...",Asignada,HOL-5INT-018,Universiteit Maastricht - Intercambio Internac...,Enero - Mayo 2015,29/AGO/2014 04:43PM,Asignada,ALE-5INT-003,2
4,14315,A00757284,Chihuahua,LMC11,84.523,78.921,Profesional,"1 - ESP-5INT-057B,<br \>3 - ESP-5INT-059,<br \...",Asignada,ARG-5INT-015,Universidad de Ciencias Empresariales y Social...,Enero - Mayo 2015,29/AGO/2014 04:43PM,Asignada,ESP-5INT-057B,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30146,588529,A01422658,Cuernavaca,LAE16,84.880,81.964,Profesional,"1 - ITA-V5SA-020M,<br \>2 - ITA-V5SA-020E,<br ...",Asignado,ITA-V5SA-020M,Università Cattolica del Sacro Cuore - Study A...,Verano 2022 Profesional,30/Mar/2022 4:33 PM,Asignado,ITA-V5SA-020M,1
30147,549639,A01561726,Chihuahua,ARQ11,84.511,109.763,Profesional,"1 - ESP-5PP-001B,<br \>2 - ESP-5INT-064A,<br \...",Admitido,ESP-5PP-001B,The Intern Group-Madrid - Prácticas Profesiona...,Agosto - Diciembre 2022,30/Mar/2022 4:34 PM,Admitido,ESP-5PP-001B,1
30148,577669,A01561940,Chihuahua,IMA11,92.024,86.391,Profesional,1 - NOR-5INT-001,Admitido,NOR-5INT-001,BI Norwegian Business School - Intercambio Int...,Agosto - Diciembre 2022,30/Mar/2022 5:10 PM,Admitido,NOR-5INT-001,1
30149,579143,A01283470,Monterrey,LDI17,87.648,82.700,Profesional,1 - ITA-5CE-016E,Asignado,ITA-5CE-016E,Scuola Lorenzo de' Medici - Certificado de Esp...,Agosto - Diciembre 2022,30/Mar/2022 5:19 PM,Asignado,ITA-5CE-016E,1


In [14]:
#Se comprueba que ya no hay datos atípicos
freq = dbase['OpcionAsignada'].value_counts() 
print(freq) 

1    27081
2     1426
3      730
4      440
5      234
6      156
7       84
Name: OpcionAsignada, dtype: int64


In [15]:
#Columna que verifica si la primera opción fue asignada o no
dbase['Primera asignada']=np.where(dbase['OportunidadAsignada']==dbase['Primeraopcion'],True, False)
dbase.head()

,Instancia,Matrícula,Campus,Programa,Promedio,PuntajeAsignación,Nivel,OportunidadesSeleccionadas,Estatus,OportunidadAsignada,Nombre Oportunidad Asignada,PeriodoAcadémico,Fecha,Estatusfiltrado,Primeraopcion,OpcionAsignada,Primera asignada
0,8453,A01139867,Monterrey,INT11,90.491,83.453,Profesional,1 - EUA-5INT-020V,Asignada,EUA-5INT-020V,California State University - Intercambio Inte...,Enero - Mayo 2015,20/AGO/2014 02:22PM,Asignada,EUA-5INT-020V,1,True
1,8289,A01018302,Santa Fe,ITC11,85.684,77.555,Profesional,"1 - SIN-5INT-003,<br \>3 - CHN-5INT-003,<br \>...",Asignada,JAP-5INT-002A,Hokkaido University - Intercambio Internaciona...,Enero - Mayo 2015,29/AGO/2014 04:34PM,Asignada,SIN-5INT-003,2,False
2,8510,A01205500,Querétaro,ISC11,89.810,85.757,Profesional,"1 - CZE-5INT-001,<br \>2 - FIN-5INT-004A,<br \...",Asignada,CZE-5INT-001,Czech Technical University - Intercambio Inte...,Enero - Mayo 2015,29/AGO/2014 04:35PM,Asignada,CZE-5INT-001,1,True
3,8007,A01139036,Monterrey,CPF11,85.465,81.528,Profesional,"1 - ALE-5INT-003,<br \>5 - ALE-5INT-034A,<br \...",Asignada,HOL-5INT-018,Universiteit Maastricht - Intercambio Internac...,Enero - Mayo 2015,29/AGO/2014 04:43PM,Asignada,ALE-5INT-003,2,False
4,14315,A00757284,Chihuahua,LMC11,84.523,78.921,Profesional,"1 - ESP-5INT-057B,<br \>3 - ESP-5INT-059,<br \...",Asignada,ARG-5INT-015,Universidad de Ciencias Empresariales y Social...,Enero - Mayo 2015,29/AGO/2014 04:43PM,Asignada,ESP-5INT-057B,7,False


In [16]:
#Se crea nueva columna con el tipo de intercambio de la opción asignada
dbase=dbase.dropna(subset=['Matrícula', 'Campus','Programa','Promedio','PuntajeAsignación','Nivel','OportunidadesSeleccionadas','Estatus','OportunidadAsignada']).reset_index(drop=True)
dbase['Tipo_intercambio']=[" ".join(re.findall('\w{2,3}-\w*[0-9]([A-Z]+)',s)) for s in dbase['OportunidadAsignada']]
dbase

,Instancia,Matrícula,Campus,Programa,Promedio,PuntajeAsignación,Nivel,OportunidadesSeleccionadas,Estatus,OportunidadAsignada,Nombre Oportunidad Asignada,PeriodoAcadémico,Fecha,Estatusfiltrado,Primeraopcion,OpcionAsignada,Primera asignada,Tipo_intercambio
0,8453,A01139867,Monterrey,INT11,90.491,83.453,Profesional,1 - EUA-5INT-020V,Asignada,EUA-5INT-020V,California State University - Intercambio Inte...,Enero - Mayo 2015,20/AGO/2014 02:22PM,Asignada,EUA-5INT-020V,1,True,INT
1,8289,A01018302,Santa Fe,ITC11,85.684,77.555,Profesional,"1 - SIN-5INT-003,<br \>3 - CHN-5INT-003,<br \>...",Asignada,JAP-5INT-002A,Hokkaido University - Intercambio Internaciona...,Enero - Mayo 2015,29/AGO/2014 04:34PM,Asignada,SIN-5INT-003,2,False,INT
2,8510,A01205500,Querétaro,ISC11,89.810,85.757,Profesional,"1 - CZE-5INT-001,<br \>2 - FIN-5INT-004A,<br \...",Asignada,CZE-5INT-001,Czech Technical University - Intercambio Inte...,Enero - Mayo 2015,29/AGO/2014 04:35PM,Asignada,CZE-5INT-001,1,True,INT
3,8007,A01139036,Monterrey,CPF11,85.465,81.528,Profesional,"1 - ALE-5INT-003,<br \>5 - ALE-5INT-034A,<br \...",Asignada,HOL-5INT-018,Universiteit Maastricht - Intercambio Internac...,Enero - Mayo 2015,29/AGO/2014 04:43PM,Asignada,ALE-5INT-003,2,False,INT
4,14315,A00757284,Chihuahua,LMC11,84.523,78.921,Profesional,"1 - ESP-5INT-057B,<br \>3 - ESP-5INT-059,<br \...",Asignada,ARG-5INT-015,Universidad de Ciencias Empresariales y Social...,Enero - Mayo 2015,29/AGO/2014 04:43PM,Asignada,ESP-5INT-057B,7,False,INT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30146,588529,A01422658,Cuernavaca,LAE16,84.880,81.964,Profesional,"1 - ITA-V5SA-020M,<br \>2 - ITA-V5SA-020E,<br ...",Asignado,ITA-V5SA-020M,Università Cattolica del Sacro Cuore - Study A...,Verano 2022 Profesional,30/Mar/2022 4:33 PM,Asignado,ITA-V5SA-020M,1,True,SA
30147,549639,A01561726,Chihuahua,ARQ11,84.511,109.763,Profesional,"1 - ESP-5PP-001B,<br \>2 - ESP-5INT-064A,<br \...",Admitido,ESP-5PP-001B,The Intern Group-Madrid - Prácticas Profesiona...,Agosto - Diciembre 2022,30/Mar/2022 4:34 PM,Admitido,ESP-5PP-001B,1,True,PP
30148,577669,A01561940,Chihuahua,IMA11,92.024,86.391,Profesional,1 - NOR-5INT-001,Admitido,NOR-5INT-001,BI Norwegian Business School - Intercambio Int...,Agosto - Diciembre 2022,30/Mar/2022 5:10 PM,Admitido,NOR-5INT-001,1,True,INT
30149,579143,A01283470,Monterrey,LDI17,87.648,82.700,Profesional,1 - ITA-5CE-016E,Asignado,ITA-5CE-016E,Scuola Lorenzo de' Medici - Certificado de Esp...,Agosto - Diciembre 2022,30/Mar/2022 5:19 PM,Asignado,ITA-5CE-016E,1,True,CE


In [17]:
#Frecuencia del tipo de intercambio
freq = dbase['Tipo_intercambio'].value_counts() 
print(freq) 

SA     18920
INT     6729
CE      1113
PP      1112
DD       853
INP      422
EI       233
PIE      229
RC       202
ID       195
PRE      119
SE        14
PEI        9
INV        1
Name: Tipo_intercambio, dtype: int64


In [18]:
#Se llama a la base de datos de campus
campus=pd.read_csv("Regiones_Campus.csv")
campus


,Campus,Region
0,Aguascalientes,Occidente
1,Chiapas,Desarrollo Regional
2,Chihuahua,Occidente
3,Ciudad de México,Ciudad de Mexico
4,Ciudad Juárez,Desarrollo Regional
5,Ciudad Obregón,Desarrollo Regional
6,Cuernavaca,Centro Sur
7,Estado de México,Ciudad de Mexico
8,Guadalajara,Occidente
9,Hidalgo​​​​​​,Centro Sur


In [19]:
#Se llama a la base de datos programas tec
programas=pd.read_csv("Programas_Tec.csv")
programas = programas.rename(columns={'Clave':'Programa'})
programas

,Programa,Programa Academico,Area Academica,Escuela
0,AMC19,Ambiente Construido Exploración,Ambiente Construido,"Arquitectura, Arte y Diseño"
1,ARQ,Arquitectura,Ambiente Construido,"Arquitectura, Arte y Diseño"
2,ARQ07,Arquitectura,Ambiente Construido,"Arquitectura, Arte y Diseño"
3,ARQ11,Arquitectura,Ambiente Construido,"Arquitectura, Arte y Diseño"
4,ARQ19,Arquitectura,Ambiente Construido,"Arquitectura, Arte y Diseño"
...,...,...,...,...
358,MC 19,Médico Cirujano,Salud,Salud
359,MO 11,Médico Odontologo,Salud,Salud
360,MO 19,Médico Odontologo,Salud,Salud
361,SLD17,Salud / Exploración,Salud,Salud


In [20]:
#Se unen las bases de datos
dbase=pd.merge(dbase,programas,on='Programa')
dbase

,Instancia,Matrícula,Campus,Programa,Promedio,PuntajeAsignación,Nivel,OportunidadesSeleccionadas,Estatus,OportunidadAsignada,...,PeriodoAcadémico,Fecha,Estatusfiltrado,Primeraopcion,OpcionAsignada,Primera asignada,Tipo_intercambio,Programa Academico,Area Academica,Escuela
0,8453,A01139867,Monterrey,INT11,90.491,83.453,Profesional,1 - EUA-5INT-020V,Asignada,EUA-5INT-020V,...,Enero - Mayo 2015,20/AGO/2014 02:22PM,Asignada,EUA-5INT-020V,1,True,INT,Ingeniería en Negocios y Tecnologías de Inform...,Ingeniería y Ciencias,Ingeniería y Ciencias
1,24386,A01152006,San Luis Potosí,INT11,85.220,77.911,Profesional,"7 - FRA-5INT-072D,<br \>4 - FRA-5INT-071B,<br ...",Asignada,FRA-5INT-071B,...,Enero - Mayo 2015,01/Sep/2014 8:37 AM,Asignada,ALE-5INT-109B,4,False,INT,Ingeniería en Negocios y Tecnologías de Inform...,Ingeniería y Ciencias,Ingeniería y Ciencias
2,32521,A01112266,Santa Fe,INT11,86.145,80.073,Profesional,"1 - ESP-5SA-059,<br \>2 - ESP-5SA-057A,<br \>3...",Asignada,ESP-5SA-059,...,Enero - Mayo 2015,02/Sep/2014 10:01 PM,Asignada,ESP-5SA-059,1,True,SA,Ingeniería en Negocios y Tecnologías de Inform...,Ingeniería y Ciencias,Ingeniería y Ciencias
3,43032,A01016931,Santa Fe,INT11,88.947,83.373,Profesional,1 - SVK-5SA-001,Asignada,SVK-5SA-001,...,Enero - Mayo 2015,24/Sep/2014 11:05 AM,Asignada,SVK-5SA-001,1,True,SA,Ingeniería en Negocios y Tecnologías de Inform...,Ingeniería y Ciencias,Ingeniería y Ciencias
4,29256,A00811718,Monterrey,INT11,85.933,81.909,Profesional,"1 - CZE-5INT-009,<br \>3 - EUA-5INT-167A,<br \...",Asignada,ALE-5INT-030D,...,Enero - Mayo 2015,30/Sep/2014 2:22 PM,Asignada,CZE-5INT-009,4,False,INT,Ingeniería en Negocios y Tecnologías de Inform...,Ingeniería y Ciencias,Ingeniería y Ciencias
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30144,520915,A01022687,Santa Fe,IRS19,83.285,111.577,Profesional,"1 - ALE-5INT-145A,<br \>2 - ALE-5INT-144A,<br ...",Admitido,FRA-5INT-050A,...,Agosto - Diciembre 2022 (Tec21),11/Mar/2022 8:44 AM,Admitido,ALE-5INT-145A,4,False,INT,Ingeniería en Robótica y Sistemas Digitales,Ingeniería y Ciencias,Ingeniería y Ciencias
30145,563908,A01365713,Querétaro,IAG19,92.500,115.483,Profesional,1 - CHL-5INT-001I,Asignado,CHL-5INT-001I,...,Agosto - Diciembre 2022 (Tec21),13/Mar/2022 5:32 PM,Asignado,CHL-5INT-001I,1,True,INT,Ingeniería en Biosistemas Agroalimentarios,Ingeniería y Ciencias,Ingeniería y Ciencias
30146,582020,A01027292,Santa Fe,LPE19,81.500,82.213,Profesional,1 - ESP-5DD-018E,Asignado,ESP-5DD-018E,...,Agosto 2022 - Junio 2023 (Tec21),23/Mar/2022 1:41 PM,Asignado,ESP-5DD-018E,1,True,DD,Licenciatura en Periodismo,Estudios Creativos,Humanidades y Educación
30147,519509,A01197317,Monterrey,LLE19,100.000,115.757,Profesional,"1 - ESP-5INT-010D,<br \>2 - ESP-5INT-081H,<br ...",Admitido,ESP-5INT-081H,...,Agosto - Diciembre 2022 (Tec21),28/Mar/2022 4:26 PM,Admitido,ESP-5INT-010D,2,False,INT,Licenciatura en Letras Hispánicas,Estudios Creativos,Humanidades y Educación


In [21]:
#Se filtran las siglas de la carrera y se le quitan los numeros de generación
dbase['Programa']=[" ".join(re.findall('[a-zA-Z]+',s)) for s in dbase['Programa']]
dbase

,Instancia,Matrícula,Campus,Programa,Promedio,PuntajeAsignación,Nivel,OportunidadesSeleccionadas,Estatus,OportunidadAsignada,...,PeriodoAcadémico,Fecha,Estatusfiltrado,Primeraopcion,OpcionAsignada,Primera asignada,Tipo_intercambio,Programa Academico,Area Academica,Escuela
0,8453,A01139867,Monterrey,INT,90.491,83.453,Profesional,1 - EUA-5INT-020V,Asignada,EUA-5INT-020V,...,Enero - Mayo 2015,20/AGO/2014 02:22PM,Asignada,EUA-5INT-020V,1,True,INT,Ingeniería en Negocios y Tecnologías de Inform...,Ingeniería y Ciencias,Ingeniería y Ciencias
1,24386,A01152006,San Luis Potosí,INT,85.220,77.911,Profesional,"7 - FRA-5INT-072D,<br \>4 - FRA-5INT-071B,<br ...",Asignada,FRA-5INT-071B,...,Enero - Mayo 2015,01/Sep/2014 8:37 AM,Asignada,ALE-5INT-109B,4,False,INT,Ingeniería en Negocios y Tecnologías de Inform...,Ingeniería y Ciencias,Ingeniería y Ciencias
2,32521,A01112266,Santa Fe,INT,86.145,80.073,Profesional,"1 - ESP-5SA-059,<br \>2 - ESP-5SA-057A,<br \>3...",Asignada,ESP-5SA-059,...,Enero - Mayo 2015,02/Sep/2014 10:01 PM,Asignada,ESP-5SA-059,1,True,SA,Ingeniería en Negocios y Tecnologías de Inform...,Ingeniería y Ciencias,Ingeniería y Ciencias
3,43032,A01016931,Santa Fe,INT,88.947,83.373,Profesional,1 - SVK-5SA-001,Asignada,SVK-5SA-001,...,Enero - Mayo 2015,24/Sep/2014 11:05 AM,Asignada,SVK-5SA-001,1,True,SA,Ingeniería en Negocios y Tecnologías de Inform...,Ingeniería y Ciencias,Ingeniería y Ciencias
4,29256,A00811718,Monterrey,INT,85.933,81.909,Profesional,"1 - CZE-5INT-009,<br \>3 - EUA-5INT-167A,<br \...",Asignada,ALE-5INT-030D,...,Enero - Mayo 2015,30/Sep/2014 2:22 PM,Asignada,CZE-5INT-009,4,False,INT,Ingeniería en Negocios y Tecnologías de Inform...,Ingeniería y Ciencias,Ingeniería y Ciencias
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30144,520915,A01022687,Santa Fe,IRS,83.285,111.577,Profesional,"1 - ALE-5INT-145A,<br \>2 - ALE-5INT-144A,<br ...",Admitido,FRA-5INT-050A,...,Agosto - Diciembre 2022 (Tec21),11/Mar/2022 8:44 AM,Admitido,ALE-5INT-145A,4,False,INT,Ingeniería en Robótica y Sistemas Digitales,Ingeniería y Ciencias,Ingeniería y Ciencias
30145,563908,A01365713,Querétaro,IAG,92.500,115.483,Profesional,1 - CHL-5INT-001I,Asignado,CHL-5INT-001I,...,Agosto - Diciembre 2022 (Tec21),13/Mar/2022 5:32 PM,Asignado,CHL-5INT-001I,1,True,INT,Ingeniería en Biosistemas Agroalimentarios,Ingeniería y Ciencias,Ingeniería y Ciencias
30146,582020,A01027292,Santa Fe,LPE,81.500,82.213,Profesional,1 - ESP-5DD-018E,Asignado,ESP-5DD-018E,...,Agosto 2022 - Junio 2023 (Tec21),23/Mar/2022 1:41 PM,Asignado,ESP-5DD-018E,1,True,DD,Licenciatura en Periodismo,Estudios Creativos,Humanidades y Educación
30147,519509,A01197317,Monterrey,LLE,100.000,115.757,Profesional,"1 - ESP-5INT-010D,<br \>2 - ESP-5INT-081H,<br ...",Admitido,ESP-5INT-081H,...,Agosto - Diciembre 2022 (Tec21),28/Mar/2022 4:26 PM,Admitido,ESP-5INT-010D,2,False,INT,Licenciatura en Letras Hispánicas,Estudios Creativos,Humanidades y Educación


In [22]:
#Se unen las bases de datos
dbase=pd.merge(dbase,campus,on='Campus')
dbase

,Instancia,Matrícula,Campus,Programa,Promedio,PuntajeAsignación,Nivel,OportunidadesSeleccionadas,Estatus,OportunidadAsignada,...,Fecha,Estatusfiltrado,Primeraopcion,OpcionAsignada,Primera asignada,Tipo_intercambio,Programa Academico,Area Academica,Escuela,Region
0,8453,A01139867,Monterrey,INT,90.491,83.453,Profesional,1 - EUA-5INT-020V,Asignada,EUA-5INT-020V,...,20/AGO/2014 02:22PM,Asignada,EUA-5INT-020V,1,True,INT,Ingeniería en Negocios y Tecnologías de Inform...,Ingeniería y Ciencias,Ingeniería y Ciencias,Norte
1,29256,A00811718,Monterrey,INT,85.933,81.909,Profesional,"1 - CZE-5INT-009,<br \>3 - EUA-5INT-167A,<br \...",Asignada,ALE-5INT-030D,...,30/Sep/2014 2:22 PM,Asignada,CZE-5INT-009,4,False,INT,Ingeniería en Negocios y Tecnologías de Inform...,Ingeniería y Ciencias,Ingeniería y Ciencias,Norte
2,55736,A01136310,Monterrey,INT,83.258,80.452,Profesional,1 - FRA-5INT-061C,Asignada,FRA-5INT-061C,...,28/Oct/2014 1:42 PM,Asignada,FRA-5INT-061C,1,True,INT,Ingeniería en Negocios y Tecnologías de Inform...,Ingeniería y Ciencias,Ingeniería y Ciencias,Norte
3,101660,A01281986,Monterrey,INT,88.086,82.745,Profesional,1 - FRA-V3SA-010H,Asignada,FRA-V3SA-010H,...,30/Mar/2015 4:15 PM,Asignada,FRA-V3SA-010H,1,True,SA,Ingeniería en Negocios y Tecnologías de Inform...,Ingeniería y Ciencias,Ingeniería y Ciencias,Norte
4,102488,A01282520,Monterrey,INT,91.365,78.514,Profesional,1 - ITA-V3SA-015M,Asignada,ITA-V3SA-015M,...,30/Mar/2015 4:58 PM,Asignada,ITA-V3SA-015M,1,True,SA,Ingeniería en Negocios y Tecnologías de Inform...,Ingeniería y Ciencias,Ingeniería y Ciencias,Norte
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29844,182993,A00517118,Ciudad Matamoros,PTM,89.738,82.653,Preparatoria,1 - CHN-V3SA-002,Asignada,CHN-V3SA-002,...,23/May/2016 5:11 PM,Asignada,CHN-V3SA-002,1,True,SA,Prepa Tec Multicultural,Prepa Tec,Prepa Tec,Desarrollo Regional
29845,182986,A00517127,Ciudad Matamoros,PTM,84.911,75.541,Preparatoria,1 - CHN-V3SA-002,Asignada,CHN-V3SA-002,...,23/May/2016 5:12 PM,Asignada,CHN-V3SA-002,1,True,SA,Prepa Tec Multicultural,Prepa Tec,Prepa Tec,Desarrollo Regional
29846,182999,A00517126,Ciudad Matamoros,PTM,88.666,82.167,Preparatoria,1 - CHN-V3SA-002,Asignada,CHN-V3SA-002,...,23/May/2016 5:14 PM,Asignada,CHN-V3SA-002,1,True,SA,Prepa Tec Multicultural,Prepa Tec,Prepa Tec,Desarrollo Regional
29847,182998,A00517125,Ciudad Matamoros,PBB,78.961,75.410,Preparatoria,1 - CHN-V3SA-002,Asignada,CHN-V3SA-002,...,23/May/2016 5:10 PM,Asignada,CHN-V3SA-002,1,True,SA,Prepa Tec Bicultural,Prepa Tec,Prepa Tec,Desarrollo Regional


In [23]:
#De la columna fecha, solo se toma el día y se excluye la hora y minutos
dbase['Fecha']=[(a[0:11]) for a in dbase['Fecha']]

#Para poder modificar el formato mes a tipo numérico, se deberán separar los datos por espacios, para que el comando replace detecte los meses mediante espacios
dbase['Fecha']=[(date.replace("/", " / ")) for date in dbase['Fecha']]
dbase

#Se crea diccionario para relacionar la forma escrita del mes con su sustituto numérico
dictOfStrings = {'ENE': '01', 'Feb':'02', 'Mar': '03', 'ABR': '04', 'May': '05', 'Jun': '06', 'Jul': '07', 'AGO': '08', 'Sep': '09', 'Oct': '10','Nov': '11','Dec': '12'}

#Se crea un ciclo para modificar fila por fila
num=0
for fecha in dbase['Fecha']:
    for word, replacement in dictOfStrings.items():
        fecha = fecha.replace(word, replacement)
    dbase.iloc[num,-10]= fecha.replace(" / ", "/")
    num+=1

dbase

,Instancia,Matrícula,Campus,Programa,Promedio,PuntajeAsignación,Nivel,OportunidadesSeleccionadas,Estatus,OportunidadAsignada,...,Fecha,Estatusfiltrado,Primeraopcion,OpcionAsignada,Primera asignada,Tipo_intercambio,Programa Academico,Area Academica,Escuela,Region
0,8453,A01139867,Monterrey,INT,90.491,83.453,Profesional,1 - EUA-5INT-020V,Asignada,EUA-5INT-020V,...,20/08/2014,Asignada,EUA-5INT-020V,1,True,INT,Ingeniería en Negocios y Tecnologías de Inform...,Ingeniería y Ciencias,Ingeniería y Ciencias,Norte
1,29256,A00811718,Monterrey,INT,85.933,81.909,Profesional,"1 - CZE-5INT-009,<br \>3 - EUA-5INT-167A,<br \...",Asignada,ALE-5INT-030D,...,30/09/2014,Asignada,CZE-5INT-009,4,False,INT,Ingeniería en Negocios y Tecnologías de Inform...,Ingeniería y Ciencias,Ingeniería y Ciencias,Norte
2,55736,A01136310,Monterrey,INT,83.258,80.452,Profesional,1 - FRA-5INT-061C,Asignada,FRA-5INT-061C,...,28/10/2014,Asignada,FRA-5INT-061C,1,True,INT,Ingeniería en Negocios y Tecnologías de Inform...,Ingeniería y Ciencias,Ingeniería y Ciencias,Norte
3,101660,A01281986,Monterrey,INT,88.086,82.745,Profesional,1 - FRA-V3SA-010H,Asignada,FRA-V3SA-010H,...,30/03/2015,Asignada,FRA-V3SA-010H,1,True,SA,Ingeniería en Negocios y Tecnologías de Inform...,Ingeniería y Ciencias,Ingeniería y Ciencias,Norte
4,102488,A01282520,Monterrey,INT,91.365,78.514,Profesional,1 - ITA-V3SA-015M,Asignada,ITA-V3SA-015M,...,30/03/2015,Asignada,ITA-V3SA-015M,1,True,SA,Ingeniería en Negocios y Tecnologías de Inform...,Ingeniería y Ciencias,Ingeniería y Ciencias,Norte
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29844,182993,A00517118,Ciudad Matamoros,PTM,89.738,82.653,Preparatoria,1 - CHN-V3SA-002,Asignada,CHN-V3SA-002,...,23/05/2016,Asignada,CHN-V3SA-002,1,True,SA,Prepa Tec Multicultural,Prepa Tec,Prepa Tec,Desarrollo Regional
29845,182986,A00517127,Ciudad Matamoros,PTM,84.911,75.541,Preparatoria,1 - CHN-V3SA-002,Asignada,CHN-V3SA-002,...,23/05/2016,Asignada,CHN-V3SA-002,1,True,SA,Prepa Tec Multicultural,Prepa Tec,Prepa Tec,Desarrollo Regional
29846,182999,A00517126,Ciudad Matamoros,PTM,88.666,82.167,Preparatoria,1 - CHN-V3SA-002,Asignada,CHN-V3SA-002,...,23/05/2016,Asignada,CHN-V3SA-002,1,True,SA,Prepa Tec Multicultural,Prepa Tec,Prepa Tec,Desarrollo Regional
29847,182998,A00517125,Ciudad Matamoros,PBB,78.961,75.410,Preparatoria,1 - CHN-V3SA-002,Asignada,CHN-V3SA-002,...,23/05/2016,Asignada,CHN-V3SA-002,1,True,SA,Prepa Tec Bicultural,Prepa Tec,Prepa Tec,Desarrollo Regional


In [24]:
#Se crea un archivo csv de la nueva base de datos
dbase.to_csv('dbase.csv')